In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [3]:
pip install seaborn

ERROR: Could not find a version that satisfies the requirement seaborn (from versions: none)
ERROR: No matching distribution found for seaborn

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import string
from string import digits
import re

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

from collections import Counter

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
data = pd.read_csv("/kaggle/input/hindienglish-corpora/Hindi_English_Truncated_Corpus.csv", encoding = 'utf-8')
data.head(10)

In [ ]:
data['source'].value_counts()

In [ ]:
data.info()

In [ ]:
data=data[data['source']=='ted']

data.head(20)

In [ ]:
data.isnull().sum()

In [ ]:
data.drop_duplicates(inplace = True)

In [ ]:
data.describe()

In [ ]:
data = data.sample(n = 30000, random_state = 42)
data.shape

In [ ]:
# Lowercase all characters
data['english_sentence']=data['english_sentence'].apply(lambda x: x.lower())
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: x.lower())

In [ ]:
# Define a function to apply text preprocessing steps
def preprocess_text(text):
    text = text.lower()
    text = re.sub("'", '', text)
    exclude = set(string.punctuation)
    text = ''.join(ch for ch in text if ch not in exclude)
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    text = re.sub("[२३०८१५७९४६]", "", text)
    text = text.strip()
    text = re.sub(" +", " ", text)
    return text

# Apply the preprocessing function to the English and Hindi sentences
data['english_sentence'] = data['english_sentence'].apply(preprocess_text)
data['hindi_sentence'] = data['hindi_sentence'].apply(preprocess_text)
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: 'START_ ' + x + ' _END')

In [ ]:
all_eng_words = set(word for eng in data['english_sentence'] for word in eng.split())
all_hindi_words = set(word for hin in data['hindi_sentence'] for word in hin.split())

print(len(all_eng_words))
all_eng_words

In [ ]:
print(len(all_hindi_words))
all_hindi_words

In [ ]:
data['len_eng_sentence'] = data['english_sentence'].apply(lambda x: len(x.split(" ")))

data['len_hindi_sentence'] = data['hindi_sentence'].apply(lambda x: len(x.split(" ")))

In [ ]:
data.head()

In [ ]:
data["eng_char_count"] = data["english_sentence"].str.len()
data["hindi_char_count"] = data["hindi_sentence"].str.len()
data["hindi_tok_count"] = data["hindi_sentence"].str.split(" ").str.len()
data["eng_tok_count"] = data["english_sentence"].str.split(" ").str.len()

In [ ]:
total_english_sentences = " "
for eng_sent in data['english_sentence'].tolist():
  total_english_sentences += eng_sent + " "

Counter(total_english_sentences.split(" ")).most_common(20)

In [ ]:
total_hindi_sentences = " "
for hindi_sent in data['hindi_sentence'].tolist():
  total_hindi_sentences += hindi_sent + " "

Counter(total_hindi_sentences.split(" ")).most_common(20)

In [ ]:
!pip install fasteda

In [ ]:
from fasteda import fast_eda

fast_eda(data[["source", "eng_char_count", "hindi_char_count", "hindi_tok_count",
               "eng_tok_count",]])

In [ ]:
data[data['len_eng_sentence']>30].shape

In [ ]:
data = data[data['len_eng_sentence']<=20]
data = data[data['len_hindi_sentence']<=20]

data

In [ ]:
print("maximum length of Hindi Sentence ",max(data['len_hindi_sentence']))
print("maximum length of English Sentence ",max(data['len_eng_sentence']))

In [ ]:
max_length_src=max(data['len_hindi_sentence'])
max_length_tar=max(data['len_eng_sentence'])

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens, num_decoder_tokens = len(input_words), len(target_words) + 1

input_token_index = {word: i + 1 for i, word in enumerate(input_words)}
target_token_index = {word: i + 1 for i, word in enumerate(target_words)}
reverse_input_char_index = {i: word for word, i in input_token_index.items()}
reverse_target_char_index = {i: word for word, i in target_token_index.items()}

num_encoder_tokens, num_decoder_tokens

In [ ]:
data = shuffle(data)
data.head(10)

In [ ]:
X, y = data['english_sentence'], data['hindi_sentence']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 42)

X_train.shape, X_test.shape

In [ ]:
# Defining a batch function:
def generate_batch(X=X_train, y=y_train, batch_size=128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),
                                          dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),
                                          dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar,
                                            num_decoder_tokens),
                                            dtype='float32')

            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size],
                                                            y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word]

                for t, word in enumerate(target_text.split()):
                    if t < len(target_text.split()) - 1:
                        decoder_input_data[i, t] = target_token_index[word]
                    if t > 0:
                        decoder_target_data[i, t - 1,
                                            target_token_index[word]] = 1.

            yield [encoder_input_data, decoder_input_data], decoder_target_data

In [ ]:
latent_dim = 300

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
model.summary()

In [ ]:
train_samples, val_samples = len(X_train), len(X_test)
batch_size, epochs = 128, 100

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['START_']
    decoded_sentence = ''
    
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '_END' or len(decoded_sentence) > 50):
            break
        
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    
    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])